# PRE PROCESS
.JPG TO .NPY

In [ ]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    for item in os.listdir(input_folder):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img_array = np.array(img)
            output_file_path = os.path.join(output_folder, os.path.splitext(item)[0] + '.npy')
            np.save(output_file_path, img_array)

input_folder_path = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train"
output_folder_path = "/kaggle/working/"

jpg_to_npy_in_folders(input_folder_path, output_folder_path)


# DOWNLOAD ZIPPED FOLDER

In [ ]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [ ]:
zip_dir()

# COUNT NUMBER OF FILES

In [ ]:
import os

def count_files(folder_path):
    # Initialize a counter for files
    num_files = 0
    
    # Iterate through all items in the folder
    for item in os.listdir(folder_path):
        # Check if the item is a file
        if os.path.isfile(os.path.join(folder_path, item)):
            # Increment the counter if it's a file
            num_files += 1
    
    return num_files

# Specify the path to the folder
folder_path = "/kaggle/working/"

# Call the function to count files
num_files = count_files(folder_path)

# Print the number of files
print("Number of files in the folder:", num_files)


In [9]:
!mv *.npy folder

In [8]:
!mkdir folder